In [2]:
import pandas as pd
import numpy as np
import datetime
import gc
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

### Clean Data

In [3]:
# read in our data
url = "https://raw.githubusercontent.com/SamIlic/Stat479_Project/master/Data/prices.csv"
df_features = pd.read_csv(url)

df_features['date'] = pd.to_datetime(df_features['date'])
df_features.head()

date Ticker       open      close        low       high     volume
0 2010-01-04    AAL   4.840000   4.770000   4.660000   4.940000    9837300
1 2010-01-04    AAP  40.700001  40.380001  40.360001  41.040001    1701700
2 2010-01-04   AAPL  30.490000  30.572857  30.340000  30.642857  123432400
3 2010-01-04    ABC  26.290001  26.629999  26.139999  26.690001    2455900
4 2010-01-04    ABT  26.000339  26.129884  25.870792  26.177866   10829000

In [92]:
"""
Create Lag Returns/Features
"""
def create_lag_features(df, lag_days):
    df_ret = df.copy()
    
    # iterate through the lag days to generate lag values up to lag_days + 1
    for i in range(1,lag_days + 2):
        df_lag = df_ret[['date', 'Ticker', 'close']].copy()
        # generate dataframe to shift index by i day. 415 is number of stocks
        df_lag['date'] = df_lag['date'].shift(-i*415)
        df_lag.columns = ['date', 'Ticker', 'value_lag' + str(i)]
        # combine the valuelag
        df_ret = pd.merge(df_ret, df_lag, how = 'left', left_on = ['date', 'Ticker'], right_on = ['date', 'Ticker'])
    
    #frees memory
    del df_lag
    
    # calculate today's percentage lag
    df_ret['Today'] = (df_ret['close'] - df_ret['value_lag1'])/(df_ret['value_lag1']) * 100.0 
    
    # calculate percentage lag
    for i in range(1, lag_days + 1):
        df_ret['lag' + str(i)] = (df_ret['value_lag'+ str(i)] - df_ret['value_lag'+ str(i+1)])/(df_ret['value_lag'+str(i+1)]) * 100.0
    
    # drop unneeded columns which are value_lags
    for i in range(1, lag_days + 2):
        df_ret.drop(['value_lag' + str(i)], axis = 1, inplace = True)
                                                                                                                                                                                                                                                                                                                                     
    return df_ret

In [93]:
df_features = create_lag_features(df_features, 5) # 5 lag features

# Sanity Check
df_temp = df_features.loc[df_features['Ticker'] == "AAPL"]
df_temp.head(7)

date Ticker       open      close        low       high     volume  \
2    2010-01-04   AAPL  30.490000  30.572857  30.340000  30.642857  123432400   
417  2010-01-05   AAPL  30.657143  30.625713  30.464285  30.798571  150476200   
832  2010-01-06   AAPL  30.625713  30.138571  30.107143  30.747143  138040000   
1247 2010-01-07   AAPL  30.250000  30.082857  29.864286  30.285715  119282800   
1662 2010-01-08   AAPL  30.042856  30.282858  29.865715  30.285715  111902700   
2077 2010-01-11   AAPL  30.400000  30.015715  29.778572  30.428572  115557400   
2492 2010-01-12   AAPL  29.884285  29.674286  29.488571  29.967142  148614900   

         Today      lag1      lag2      lag3      lag4      lag5  
2          NaN       NaN       NaN       NaN       NaN       NaN  
417   0.172887       NaN       NaN       NaN       NaN       NaN  
832  -1.590633  0.172887       NaN       NaN       NaN       NaN  
1247 -0.184858 -1.590633  0.172887       NaN       NaN       NaN  
1662  0.664833 -0.184858 -1.590633  0.172887       NaN       NaN  
2077 -0.882160  0.664833 -0.184858 -1.590633  0.172887       NaN  
2492 -1.137500 -0.882160  0.664833 -0.184858 -1.590633  0.172887

In [12]:
# drop earlier data with missing lag features
df_features.dropna(inplace=True)

# finds all unique stocks
uniq_stocks = set(df_features['Ticker'])

### DT Model

#### Train-Test Split

READ THIS - https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/

1) __Train-Test__ split that respect temporal order of observations.  
2) __Multiple Train-Test__ splits that respect temporal order of observations.  
3) __Walk-Forward Validation__ where a model may be updated each time step new data is received.  

In [81]:
# training data is anything less than 2016
X_train = df_features.loc[(df_features['date'] < '2016-01-01'),:]
y_train = X_train['Today']

# testing data is 2016
X_test = df_features.loc[(df_features['date'] > '2015-12-31'), :]
y_test = X_test['Today']

### Drop columns
## Train
# sam might need these column for feature engineering
X_train.drop(['open','close','low','high','Today'], axis = 1, inplace = True)
## Test
# sam might need these column for feature engineering
X_test.drop(['open','close','low','high','Today'], axis = 1, inplace = True)

In [82]:
X_train.head(7)

date Ticker     volume      lag1      lag2      lag3      lag4  \
2490 2010-01-19    AAL    7807700 -1.610018  2.007299  7.874016  0.793651   
2491 2010-01-19    AAP    1295000 -0.631951 -1.322023  1.390994 -1.739565   
2492 2010-01-19   AAPL  182501900 -1.671203 -0.579161  1.410553 -1.137500   
2493 2010-01-19    ABC    2826200 -1.065781  0.628691  1.960788  0.683371   
2494 2010-01-19    ABT   16663600  0.252932  0.108517  0.217513 -0.289176   
2495 2010-01-19   ADBE    5964100 -0.083574 -1.047401  1.738640 -1.518915   
2496 2010-01-19    ADI    3906700 -3.370037 -0.100003 -0.033322 -4.152025   
2497 2010-01-19    ADM    4336000 -0.650615  0.260926  0.425807 -1.069345   
2498 2010-01-19    ADS    1032700 -1.574453  0.105072 -0.030006 -1.638378   
2499 2010-01-19   ADSK    2395100 -1.954887  1.333333  1.390502 -1.371432   

          lag5  
2490 -1.945525  
2491 -0.984245  
2492 -0.882160  
2493  1.113248  
2494  0.508629  
2495 -1.308258  
2496 -0.571613  
2497  0.064854  
2498  0.103428  
2499 -0.038081

In [83]:
# loop through stock data here for additional features [MAY BE PICK ONLY 100/400 stocks???]
# Final model we should use say 100
# when training, for the purpose of time, we should use <=10
Ticker = "AAPL"
X_train_one = X_train.loc[X_train['Ticker'] == Ticker, :]
y_train_one_list = list(y_train[X_train_one.index])
X_train_one.drop(['Ticker', 'date'], axis = 1, inplace = True)

X_test_one = X_test.loc[X_test['Ticker'] == Ticker, :]
y_test_one_list = list(y_test[X_test_one.index])
X_test_one.drop(['Ticker', 'date'], axis = 1, inplace = True)

#### MSE

In [95]:
# random forest regression
rf = RandomForestRegressor(random_state = 0, n_jobs = 1, n_estimators = 10)
rf.fit(X_train_one, y_train_one_list)
print(mean_squared_error(y_test_one_list, rf.predict(X_test_one)))

2.44501189116485


#### R2

In [96]:
from sklearn.metrics import r2_score
# r2_score(y_test_one.values.astype(np.float), rf.predict(X_test_one.values.astype(np.float)))
r2_score(y_test_one_list, rf.predict(X_test_one))

-0.13264689085616932

In [109]:
y_test_one_list
#df = pd.DataFrame([y_test_one_list,rf.predict(X_test_one)],columns=['y_test_one_list','rf.predict(X_test_one)'],dtype=float)
TTT = pd.DataFrame(y_test_one_list, columns=['y_test_one_list'])
TTT['X_test_one'] = rf.predict(X_test_one)
#TTT.head(20)

### Correlation
print("Cor: ", np.corrcoef(TTT['y_test_one_list'], TTT['X_test_one'])[0, 1])
print("R2: ", r2_score(TTT['y_test_one_list'], TTT['X_test_one']))

#fdmg

Cor:  -0.023104325001244184
R2:  -0.13264689085616932


In [28]:
import matplotlib.pyplot as plt

a = df_features.loc[df_features.loc[:,'Ticker'] == 'AAPL', ['date', 'close','volume']]
a.plot(x = 'date', y = 'close', title = "Apple - Returns")
a.plot(x = 'date', y = 'volume', title = "Apple - Returns")
plt.show()